# MS Office in Python

### python-docx for Word.

In [5]:
from docx import Document

# Create a new Document
doc = Document()

# Define the tests and their results
tests = ['Success', 'Success', 'Fail', 'Success', 'Success', 'Fail', 'Fail', 'Success', 'Success']

# Write the tests and their results to the document
for i, result in enumerate(tests, start=1):
    doc.add_paragraph(f'Task {i}: {result}')

# Save the document
try:
    doc.save('task_results.docx')
    print('File saved successfully.')
except PermissionError:
    print('Please close the file first.')

File saved successfully.


### openpyxl for Excel.

In [6]:
from openpyxl import Workbook

# Create a new Workbook
wb = Workbook()

# Select active worksheet
ws = wb.active

# Define the headers and the data
headers = ['Task 1', 'Task 2', 'Task 3', 'Task 4', 'Task 5', 'Task 6', 'Task 7', 'Task 8', 'Task 9']
data = [
    ['x', 'x', None, 'x', 'x', 'x', None, None, 'x'],
    ['x', None, None, None, None, None, 'x', 'x', 'x'],
    ['x', 'x', 'x', 'x', 'x', None, None, None, 'x'],
]

# Write the headers and the data to the worksheet
ws.append([''] + headers)
for i, row in enumerate(data, start=1):
    ws.append([f'Test {i}'] + row)

# Save the workbook
try:
    wb.save("versions_testing.xlsx")
    print('File saved successfully.')
except PermissionError:
    print('Please close the file first.')

File saved successfully.


### python-pptx for PowerPoint.

In [7]:
from openpyxl import load_workbook
from docx import Document
from pptx import Presentation
from pptx.util import Inches

'''
collect the tests and tasks from the Excel file
'''
wb = load_workbook('versions_testing.xlsx')
ws = wb.active
tests = {}
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    tasks = []
    for cell in row:
        # NOTICE: in python if the first check is false, the second one is not evaluated
        if cell.value is not None and cell.value.lower() == 'x':
            tasks.append(str(cell.column - 1))
    tests[row[0].value] = tasks

'''
collect the paragraphs from the document
'''
doc = Document('task_results.docx')
test_paragraphs = {}
for i, p in enumerate(doc.paragraphs):
    test_name = "Task " + str(i + 1)
    test_paragraphs[test_name] = p.text

'''
create a presentation for each test
'''
for test, tasks in tests.items():
    presentation = Presentation()

    # Add a slide with a title and content layout
    slide_layout = presentation.slide_layouts[5] # chose this because layout 5 has a title only
    slide = presentation.slides.add_slide(slide_layout)

    # Add a title to the slide
    title = slide.shapes.title
    title.text = f"{test} Results"

    # Filter tests based on Excel file
    filtered_tests = [test_paragraphs[f"Task {task}"] for task in tasks]
    '''
    Same thing without list comprehension:
    filtered_tests = []
    for task in tasks:
        test_key = f"Test {task}"
        test_value = test_paragraphs[test_key]
        filtered_tests.append(test_value)
    '''

    # Add a table to the slide with an additional row for headers
    rows, cols = len(filtered_tests) + 1, 2

    # NOTICE: we can assign values to several variables this way in one line
    left = top = Inches(3.0)
    width = height = Inches(4.0)

    # add table to slide
    table = slide.shapes.add_table(rows, cols, left, top, width, height).table

    # Add headers to the table 
    table.cell(0, 0).text = "Task name"
    table.cell(0, 1).text = "Result"

    # Fill the table with the tests from the Word document
    for i, filtered_test in enumerate(filtered_tests, start=1):  # start enumeration from 1 to skip headers
        test_name, result = filtered_test.split(": ")
        table.cell(i, 0).text = test_name
        table.cell(i, 1).text = result

    # Save the presentation
    try:
        formatted_test_name = test.replace(' ', '_').lower() # Just to make the file name look nicer
        presentation.save(f"{formatted_test_name}_presentation.pptx")
        print(f"Presentation for {test} saved successfully.")
    except PermissionError:
        print(f"Presentation {test} is open, you have to close it first!")

Presentation for Test 1 saved successfully.
Presentation for Test 2 saved successfully.
Presentation for Test 3 saved successfully.
